In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import json
from omegaconf.omegaconf import OmegaConf, open_dict
import shutil

from nemo.collections.tts.models.speechllm.megatron_t5_speechllm_model import MegatronT5SpeechLMModel
from nemo.collections.nlp.parts.megatron_trainer_builder import MegatronTrainerBuilder
from nemo.collections.asr.parts.preprocessing.segment import AudioSegment
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager
from IPython.display import Audio, display
import torchaudio

# CHANGE THIS TO A LOCAL DIRECTORY
EXP_DIR = "/datap/misc/NotebookInference"

if not os.path.exists(EXP_DIR):
    os.makedirs(EXP_DIR)

## Save a dummy manifest to setup Model Test Step

In [2]:
def write_records(fp, records):
    with open(fp, "w") as f:
        for record in records:
            f.write(json.dumps(record) + "\n")

dummy_codes = torch.ones(8, 300).cpu().type(torch.int16)
dummy_codes_fp = os.path.join(EXP_DIR, "dummy_codes.pt")
torch.save(dummy_codes, dummy_codes_fp)


dummy_record = {
    "question" : "Phoneme TTS Sample Text",
    "answer" : dummy_codes_fp,
    "context" : dummy_codes_fp,
    "context_type" : "REFSPEAKERCODEC",
    "question_type" : "TEXT",
    "answer_type" : "AUDIOCODEC",
    "context_duration" : 5.0,
    "answer_duration" : 5.0,
    "taskname" : "squad"
}

dummy_val_file = os.path.join(EXP_DIR, "dummy_val.json")

write_records(dummy_val_file, [dummy_record])

## Load and setup the model

In [17]:
# CHANGE THESE PATHS TO RELEVANT MOUNTED PATHS IN DOCKER
config_path = "/home/pneekhara/2023/NeMo/examples/tts/speechllm/conf/megatron_t5_speechllm_inference_multiencoder.yaml"
# checkpoint_path = "/datap/misc/temp_checkpoints_new/desta_less_sophia_highLR_step159600.ckpt"
checkpoint_path = "/datap/misc/productionlevel21HzCkpts/multiencoder_step135k.ckpt"
codecmodel_path = "/datap/misc/checkpoints/AudioCodec_21Hz-2k-codes_updated.nemo"
vocab_file = "/datap/misc/checkpoints/9a77f10c2793465e8e8a3fa5fcbef8b0_vocab.txt"

cfg = OmegaConf.load(config_path)

if "gradient_as_bucket_view" not in cfg.model:
    with open_dict(cfg):
        cfg.model.gradient_as_bucket_view=False

trainer = MegatronTrainerBuilder(cfg).create_trainer()
exp_manager(trainer, cfg.exp_manager)

with open_dict(cfg):
    cfg.exp_manager.exp_dir=EXP_DIR
    cfg.checkpoint_path = checkpoint_path
    cfg.model.data.sup_data_path="/datap/misc/speechllm_codecdatasets/"
    cfg.model.global_batch_size=1
    cfg.model.micro_batch_size=1
    cfg.model.data.speech_offset=30128
    cfg.model.lm_vocab_size=30000
    cfg.model.data.add_special_tokens_to_only_first_codebook=True
    cfg.model.data.train_task="all"
    cfg.model.freeze_model=False
    cfg.model.data.max_seq_length=512
    cfg.model.data.min_seq_length=10
    cfg.model.data.codebook_fps=21
    cfg.model.data.speech_codebook_size=2048
    cfg.model.max_inference_timesteps=512
    cfg.model.data.context_duration_min=20.0
    cfg.model.data.context_duration_max=20.0
    cfg.model.top_k=80
    cfg.model.temperature=0.85
    cfg.model.data.speech_offset=30128
    cfg.model.lm_vocab_size=30000
    cfg.model.codecmodel_path=codecmodel_path
    cfg.trainer.devices=1
    cfg.trainer.precision="bf16"
    cfg.model.precision = cfg.trainer.precision
    cfg.model.override_tokenizer_vocab_file=vocab_file
    cfg.model.english_only_model=True
    cfg.model.asr_model_name="stt_en_conformer_transducer_large"
    cfg.model.frozen_model.decoder.layer_type=[1,1,1,2,2,2,2,2,2,2,1,1]
    cfg.model.alignment_decoder_layerids=[0,1,2,3,4]
    cfg.model.enc_output_to_layers=[[8,9],[3,4,5,6,7]]
    cfg.model.data.test_ds=[dummy_val_file]
    cfg.model.data.num_workers = 0
    cfg.model.use_alignment_loss=True
    cfg.model.return_all_crossattention_probs=True


checkpoint_path = cfg.get('checkpoint_path', None)
assert checkpoint_path is not None, "checkpoint path needs to be valid"

model = MegatronT5SpeechLMModel.load_from_checkpoint(
        checkpoint_path=checkpoint_path, trainer=trainer, cfg=cfg.model
    )
model.eval()
model = model.cuda()

codec_model = model.additional_models['codec']
trainer.test(model)


[NeMo I 2024-09-24 19:14:47 megatron_trainer_builder:51] Detected interactive environment, using NLPDDPStrategyNotebook


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo W 2024-09-24 19:14:47 exp_manager:780] No version folders would be created under the log folder as 'resume_if_exists' is enabled.
[NeMo W 2024-09-24 19:14:47 exp_manager:636] There were no checkpoints found in checkpoint_dir or no checkpoint folder at checkpoint_dir :/home/pneekhara/2023/NeMo/nemo_experiments/megatron_t5_speechllm/checkpoints. Training from scratch.


[NeMo I 2024-09-24 19:14:47 exp_manager:402] Experiments will be logged at /home/pneekhara/2023/NeMo/nemo_experiments/megatron_t5_speechllm
[NeMo I 2024-09-24 19:14:47 exp_manager:863] TensorboardLogger has been set up


[NeMo W 2024-09-24 19:14:47 exp_manager:973] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to 250000. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.
[NeMo W 2024-09-24 19:14:47 nemo_model_checkpoint:58] Found save_best_model is True and save_nemo_on_train_end is False. Set save_nemo_on_train_end to True to automatically save the best model.
[NeMo W 2024-09-24 19:14:48 megatron_base_model:1078] The model: MegatronT5SpeechLMModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-09-24 19:14:48 megatron_base_model:1078] The model: MegatronT5SpeechLMModel() does not have field.name: virtual_pipeline_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-09-24 19:14:48 megatron_base_model:1078] The model: MegatronT5SpeechLMMode

[NeMo I 2024-09-24 19:14:48 megatron_init:241] Rank 0 has data parallel group : [0]
[NeMo I 2024-09-24 19:14:48 megatron_init:247] Rank 0 has combined group of data parallel and context parallel : [0]
[NeMo I 2024-09-24 19:14:48 megatron_init:252] All data parallel group ranks with context parallel combined: [[0]]
[NeMo I 2024-09-24 19:14:48 megatron_init:255] Ranks 0 has data parallel rank: 0
[NeMo I 2024-09-24 19:14:48 megatron_init:272] Rank 0 has context parallel group: [0]
[NeMo I 2024-09-24 19:14:48 megatron_init:275] All context parallel group ranks: [[0]]
[NeMo I 2024-09-24 19:14:48 megatron_init:276] Ranks 0 has context parallel rank: 0
[NeMo I 2024-09-24 19:14:48 megatron_init:287] Rank 0 has model parallel group: [0]
[NeMo I 2024-09-24 19:14:48 megatron_init:288] All model parallel group ranks: [[0]]
[NeMo I 2024-09-24 19:14:48 megatron_init:298] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-09-24 19:14:48 megatron_init:302] All tensor model parallel group ranks: 

...

[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test titanet_avg_cossim: -0.0505669042468071
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test titanet_avg_cossim_context_pred: -0.0505669042468071
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test titanet_avg_cossim_context_gt: 1.0000001192092896
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test wavlm_avg_cossim: 0.4690419137477875
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test wavlm_avg_cossim_context_pred: 0.4690573811531067
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test wavlm_avg_cossim_context_gt: 1.0
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test squim_mos_pred_GT: 2.735186815261841
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test squim_mos_GT_context: 3.95481276512146
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test squim_mos_pred_context: 2.6783905029296875
[NeMo I 2024-09


[NeMo W 2024-09-24 19:16:04 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
      return _methods._mean(a, axis=axis, dtype=dtype,
    
[NeMo W 2024-09-24 19:16:04 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
      ret = ret.dtype.type(ret / rcount)
    


[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test cer_tts: nan
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test wer_tts: nan
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test cer_transcript_gt: 0.09090909090909091
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test wer_transcript_gt: 0.5
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test cer_phoneme_gt: 0.09090909090909091
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test wer_phoneme_gt: 0.5
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test cer_tts_gt: nan
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test wer_tts_gt: nan
[NeMo I 2024-09-24 19:16:04 megatron_t5_speechllm_model:1383] Test RTF: 0.7665415829251288


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric              ┃             DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│               test_RTF               │          0.766541600227356           │
│           test_cer_phoneme           │                 inf                  │
│         test_cer_phoneme_gt          │         0.09090909361839294          │
│         test_cer_transcript          │                 inf                  │
│        test_cer_transcript_gt        │         0.09090909361839294          │
│             test_cer_tts             │                 nan                  │
│           test_cer_tts_gt            │                 nan                  │
│      test_squim_mos_GT_context       │           3.95481276512146           │
│        test_squim_mos_pred_GT        │          2.735186815261841           │
│     test_squim_mos_pred_context      │          2.6783905029296875          │
│       test_titanet_avg_cossim        │         -0.0505669042468071          │
│  test_titanet_avg_cossim_context_gt  │          1.0000001192092896          │
│ test_titanet_avg_cossim_context_pred │         -0.0505669042468071          │
│        test_wavlm_avg_cossim         │          0.4690419137477875          │
│   test_wavlm_avg_cossim_context_gt   │                 1.0                  │
│  test_wavlm_avg_cossim_context_pred  │          0.4690573811531067          │
│           test_wer_phoneme           │                 inf                  │
│         test_wer_phoneme_gt          │                 0.5                  │
│         test_wer_transcript          │                 inf                  │
│        test_wer_transcript_gt        │                 0.5                  │
│             test_wer_tts             │                 nan                  │
│           test_wer_tts_gt            │                 nan                  │
└──────────────────────────────────────┴──────────────────────────────────────┘

[{'test_titanet_avg_cossim': -0.0505669042468071,
  'test_titanet_avg_cossim_context_pred': -0.0505669042468071,
  'test_titanet_avg_cossim_context_gt': 1.0000001192092896,
  'test_wavlm_avg_cossim': 0.4690419137477875,
  'test_wavlm_avg_cossim_context_pred': 0.4690573811531067,
  'test_wavlm_avg_cossim_context_gt': 1.0,
  'test_squim_mos_pred_GT': 2.735186815261841,
  'test_squim_mos_GT_context': 3.95481276512146,
  'test_squim_mos_pred_context': 2.6783905029296875,
  'test_cer_transcript': inf,
  'test_wer_transcript': inf,
  'test_cer_phoneme': inf,
  'test_wer_phoneme': inf,
  'test_cer_tts': nan,
  'test_wer_tts': nan,
  'test_cer_transcript_gt': 0.09090909361839294,
  'test_wer_transcript_gt': 0.5,
  'test_cer_phoneme_gt': 0.09090909361839294,
  'test_wer_phoneme_gt': 0.5,
  'test_cer_tts_gt': nan,
  'test_wer_tts_gt': nan,
  'test_RTF': 0.766541600227356}]

## Helper functions

In [4]:
out_dir = os.path.join( model.trainer.logger.save_dir, model.trainer.logger.name, model.trainer.logger.version, "Sample_Audios")
out_path = os.path.join(out_dir, 'predicted_wav_0.wav')


def encode(wav_path):
    # Convert an audio file to nemo codec codes
    features = AudioSegment.segment_from_file(
                    wav_path, target_sr=codec_model.sample_rate, n_segments=-1, trim=False,
                )
    audio_samples = features.samples
    audio = torch.tensor(audio_samples).cuda()
    audio_length = torch.tensor(audio.size(0)).long().cuda()
    print(f"audio {audio.size()} audio_length {audio_length}")
    print(f"audio {audio.device} audio_length {audio_length.device} codec_model {codec_model.device}")

    original_codec_codes, _ = codec_model.encode(audio=audio.unsqueeze(0), audio_len=audio_length.unsqueeze(0))
    original_codec_codes = original_codec_codes[0]
    print(f"original_codec_codes {original_codec_codes.size()} audio {audio.size()} audio_length {audio_length}")
    duration = original_codec_codes.size()[1] / 21
    
    target_codec_filepath = wav_path[:-4] + "_codes.pt"
    torch.save(original_codec_codes.cpu().type(torch.int16), target_codec_filepath)
    return original_codec_codes, target_codec_filepath, duration
    
    
    
def play_codec(codec_path):
    # Convert nemo codecs to audio and play it
    codec = torch.load(codec_path)
    codec = codec.to('cuda')
    codec = codec.unsqueeze(0)
    codec_lens = torch.Tensor([codec.shape[2]]).long().cuda()
    codec_decoded_audios, _ = codec_model.decode(tokens=codec.long(), tokens_len=codec_lens)
    codec_decoded_audio = codec_decoded_audios[0]
    temp_wav_path = os.path.join(EXP_DIR, "temp.wav")
    torchaudio.save(temp_wav_path, codec_decoded_audio[None].cpu(), 22050)
    display(Audio(temp_wav_path))

def generate_new_audio(
    text,
    context,
    context_duration=4.0,
    context_type="REFSPEAKERCODEC",
    temperature=0.85,
    top_k=80,
    text_task="Phoneme TTS "
    ):
    # Prepare data in speechllm format
    model.cfg.temperature = temperature
    model.cfg.top_k = top_k
    dummy_answer = dummy_codes_fp
    json_in = {}
    json_in["question"] = text_task + text
    json_in["question_type"] = "TEXT"
    json_in["answer"] = dummy_answer 
    json_in["context"] = context 
    json_in["answer_type"] = "AUDIOCODEC"
    json_in["context_type"] = context_type
    json_in["context_duration"] = context_duration
    json_in["answer_duration"] = 2.0
    json_in["taskname"] = "squad"
    json_in["lang"] = "en"
    json_in = [json_in]
    
    # Prepare dataloader
    model._test_ds.examples = []
    model._test_ds.examples = model._test_ds.load_data(json_in)
    
    sampler = torch.utils.data.distributed.DistributedSampler(
            model._test_ds, num_replicas=1, rank=0, shuffle=False, seed=1
        )

    model._test_dl = torch.utils.data.DataLoader(
        model._test_ds,
        collate_fn=model._test_ds.collate_fn,
        sampler=sampler,
        batch_size=1,
        drop_last=False,
        num_workers=1,
        pin_memory=False,
        persistent_workers=True
    )
    
    # Run inference
    model.cfg.data.test_ds = None
    trainer.test(model, model._test_dl)
    print("Out path:", out_path)
    print("Inference done")

In [5]:
text_contexts = [
    "TEXT CONTEXT: | Language:en Dataset:Riva Speaker:Lindy_CMU_FEARFUL |",
    "TEXT CONTEXT: | Language:en Dataset:Riva Speaker:Lindy_CMU_HAPPY |",
    "TEXT CONTEXT: | Language:en Dataset:Riva Speaker:Rodney_CMU_HAPPY |",
    "TEXT CONTEXT: | Language:en  Dataset:PromptTTS Gender:female SpeakingRate:2. Slow emotion:neutral Pitch:4. High SNR:5. Clean REVERB:5. Very close-sounding |"
]

## Generate audio from a text context

In [16]:
text = "As I closed my laptop for the night, my reflection in the screen continued to smile back at me."
text_task = "Phoneme TTS " # Can be "Text to speech this " (for sentence-piece tokenizer) or "Phoneme TTS " (for phoneme tokenizer)
context = text_contexts[0] # Sample Text Context
context_type = "TEXT" # Can be REFSPEAKERCODEC (for audio context), TEXT (for text context)
generate_new_audio(
    text, 
    context, 
    context_type=context_type, 
    context_duration=5.0, # Does not matter, should just be > 3 so that dataset does not filter it out.
    top_k=80, # Can play around with this to check roubstness
    temperature=0.85, # Can play around with this. temperature < 0.85 can be more robust
    text_task=text_task
)
display(Audio(out_path))

[NeMo I 2024-09-24 19:11:33 t5_speechllm_dataset:341] copy_dataset len === 1


  0%|          | 0/1 [00:00<?, ?it/s]

[NeMo I 2024-09-24 19:11:33 t5_speechllm_dataset:441] Skipped 0 sentences, sequence length too short or too long even after truncation


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2024-09-24 19:11:34 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


Testing: 0it [00:00, ?it/s]

[NeMo I 2024-09-24 19:11:36 megatron_t5_speechllm_model:1590] Timestep 100
[NeMo I 2024-09-24 19:11:36 megatron_t5_speechllm_model:1698] End detected for item 0 at timestep 115
All ends detected
[NeMo I 2024-09-24 19:11:37 megatron_t5_speechllm_model:1864] Clipping until end index for audio 0


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
[NeMo W 2024-09-24 19:11:38 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5083: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
      warnings.warn(
    

Transcribing: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.68it/s]
[NeMo W 2024-09-24 19:11:38 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
      return _methods._mean(a, axis=axis, dtype=dtype,
    
[NeMo W 2024-09-24 19:11:38 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
      ret = ret.dtype.type(ret / rcount)
    


[NeMo I 2024-09-24 19:11:38 megatron_t5_speechllm_model:1383] Test titanet_avg_cossim: 0.00905416626483202
[NeMo I 2024-09-24 19:11:38 megatron_t5_speechllm_model:1383] Test titanet_avg_cossim_context_pred: nan
[NeMo I 2024-09-24 19:11:38 megatron_t5_speechllm_model:1383] Test titanet_avg_cossim_context_gt: nan
[NeMo I 2024-09-24 19:11:38 megatron_t5_speechllm_model:1383] Test wavlm_avg_cossim: 0.38731834292411804
[NeMo I 2024-09-24 19:11:38 megatron_t5_speechllm_model:1383] Test wavlm_avg_cossim_context_pred: nan
[NeMo I 2024-09-24 19:11:38 megatron_t5_speechllm_model:1383] Test wavlm_avg_cossim_context_gt: nan
[NeMo I 2024-09-24 19:11:38 megatron_t5_speechllm_model:1383] Test squim_mos_pred_GT: 2.197134321386164
[NeMo I 2024-09-24 19:11:38 megatron_t5_speechllm_model:1383] Test squim_mos_GT_context: nan
[NeMo I 2024-09-24 19:11:38 megatron_t5_speechllm_model:1383] Test squim_mos_pred_context: nan
[NeMo I 2024-09-24 19:11:38 megatron_t5_speechllm_model:1383] Test cer_transcript: inf
[

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric              ┃             DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│               test_RTF               │          0.6132990717887878          │
│           test_cer_phoneme           │                 inf                  │
│         test_cer_phoneme_gt          │         0.003225806402042508         │
│         test_cer_transcript          │                 inf                  │
│        test_cer_transcript_gt        │        0.0029325513169169426         │
│             test_cer_tts             │                 inf                  │
│           test_cer_tts_gt            │                 0.0                  │
│      test_squim_mos_GT_context       │                 nan                  │
│        test_squim_mos_pred_GT        │          2.197134256362915           │
│     test_squim_mos_pred_context      │                 nan                  │
│       test_titanet_avg_cossim        │         0.00905416626483202          │
│  test_titanet_avg_cossim_context_gt  │                 nan                  │
│ test_titanet_avg_cossim_context_pred │                 nan                  │
│        test_wavlm_avg_cossim         │         0.38731834292411804          │
│   test_wavlm_avg_cossim_context_gt   │                 nan                  │
│  test_wavlm_avg_cossim_context_pred  │                 nan                  │
│           test_wer_phoneme           │                 inf                  │
│         test_wer_phoneme_gt          │         0.010526316240429878         │
│         test_wer_transcript          │                 inf                  │
│        test_wer_transcript_gt        │         0.009569377638399601         │
│             test_wer_tts             │                 inf                  │
│           test_wer_tts_gt            │                 0.0                  │
└──────────────────────────────────────┴──────────────────────────────────────┘

Out path: /home/pneekhara/2023/NeMo/nemo_experiments/megatron_t5_speechllm/Sample_Audios/predicted_wav_0.wav
Inference done


## Listen to some ground-truth context audios

In [7]:
context_paths = [
    "/datap/misc/speechllm_codecdatasets/codecs/RivattsEnglish21Hz_nemo_codec21_bw_6.0/target_codes_Rodney_22khz_TASKMASTER_RODNEY_TASKMASTER_000578.pt",
]

for cidx, context_path in enumerate(context_paths):
    print(cidx, context_path)
    play_codec(context_path)

0 /datap/misc/speechllm_codecdatasets/codecs/RivattsEnglish21Hz_nemo_codec21_bw_6.0/target_codes_Rodney_22khz_TASKMASTER_RODNEY_TASKMASTER_000578.pt


## Generate audio from an audio context

In [8]:
text = "As I closed my laptop for the night, my reflection in the screen continued to smile back at me."
text_task = "Text to speech this " # Can be "Text to speech this " (for sentence-piece tokenizer) or "Phoneme TTS " (for phoneme tokenizer)
context = context_paths[0] # Sample Text Context
context_type = "REFSPEAKERCODEC" # Can be REFSPEAKERCODEC (for audio context), TEXT (for text context)
generate_new_audio(
    text, 
    context, 
    context_type=context_type, 
    context_duration=5.0, # Does not matter, should just be > 3 so that dataset does not filter it out.
    temperature=0.8, # Can play around with this. temperature < 0.85 can be more robust
    text_task=text_task
)
display(Audio(out_path))

[NeMo I 2024-09-24 18:49:48 t5_speechllm_dataset:341] copy_dataset len === 1


  0%|          | 0/1 [00:00<?, ?it/s]

[NeMo I 2024-09-24 18:49:48 t5_speechllm_dataset:441] Skipped 0 sentences, sequence length too short or too long even after truncation


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2024-09-24 18:49:48 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


Testing: 0it [00:00, ?it/s]

[NeMo I 2024-09-24 18:49:51 megatron_t5_speechllm_model:1590] Timestep 100
[NeMo I 2024-09-24 18:49:51 megatron_t5_speechllm_model:1698] End detected for item 0 at timestep 115
All ends detected
[NeMo I 2024-09-24 18:49:52 megatron_t5_speechllm_model:1864] Clipping until end index for audio 0


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
[NeMo W 2024-09-24 18:49:53 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5083: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
      warnings.warn(
    
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.

Transcribing: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.64it/s]

[NeMo I 2024-09-24 18:49:53 megatron_t5_speechllm_model:1383] Test titanet_avg_cossim: 0.00466860830783844
[NeMo I 2024-09-24 18:49:53 megatron_t5_speechllm_model:1383] Test titanet_avg_cossim_context_pred: nan
[NeMo I 2024-09-24 18:49:53 megatron_t5_speechllm_model:1383] Test titanet_avg_cossim_context_gt: nan
[NeMo I 2024-09-24 18:49:53 megatron_t5_speechllm_model:1383] Test wavlm_avg_cossim: 0.5286357998847961
[NeMo I 2024-09-24 18:49:53 megatron_t5_speechllm_model:1383] Test wavlm_avg_cossim_context_pred: nan
[NeMo I 2024-09-24 18:49:53 megatron_t5_speechllm_model:1383] Test wavlm_avg_cossim_context_gt: nan
[NeMo I 2024-09-24 18:49:53 megatron_t5_speechllm_model:1383] Test squim_mos_pred_GT: 2.1462229092915854
[NeMo I 2024-09-24 18:49:53 megatron_t5_speechllm_model:1383] Test squim_mos_GT_context: nan
[NeMo I 2024-09-24 18:49:53 megatron_t5_speechllm_model:1383] Test squim_mos_pred_context: nan
[NeMo I 2024-09-24 18:49:53 megatron_t5_speechllm_model:1383] Test cer_transcript: inf
[

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric              ┃             DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│               test_RTF               │          0.755886971950531           │
│           test_cer_phoneme           │                 inf                  │
│         test_cer_phoneme_gt          │         0.005376344081014395         │
│         test_cer_transcript          │                 inf                  │
│        test_cer_transcript_gt        │         0.00358422938734293          │
│             test_cer_tts             │                 inf                  │
│           test_cer_tts_gt            │                 0.0                  │
│      test_squim_mos_GT_context       │                 nan                  │
│        test_squim_mos_pred_GT        │          2.1462228298187256          │
│     test_squim_mos_pred_context      │                 nan                  │
│       test_titanet_avg_cossim        │         0.00466860830783844          │
│  test_titanet_avg_cossim_context_gt  │                 nan                  │
│ test_titanet_avg_cossim_context_pred │                 nan                  │
│        test_wavlm_avg_cossim         │          0.5286357998847961          │
│   test_wavlm_avg_cossim_context_gt   │                 nan                  │
│  test_wavlm_avg_cossim_context_pred  │                 nan                  │
│           test_wer_phoneme           │                 inf                  │
│         test_wer_phoneme_gt          │         0.02631578966975212          │
│         test_wer_transcript          │                 inf                  │
│        test_wer_transcript_gt        │         0.017543859779834747         │
│             test_wer_tts             │                 inf                  │
│           test_wer_tts_gt            │                 0.0                  │
└──────────────────────────────────────┴──────────────────────────────────────┘

Out path: /home/pneekhara/2023/NeMo/nemo_experiments/megatron_t5_speechllm/Sample_Audios/predicted_wav_0.wav
Inference done
